# OP-WORKFLOW-CAGEscan-short-reads-v2.0

This document is an example of how to process a C1CAGE library with a Jupyter notebook from raw reads to single molecule count. All the steps are described in the [tutorial](https://github.com/Population-Transcriptomics/C1-CAGE-preview/blob/master/tutorial.md) section of this repository. In the following section we assume that:
- you are familiar with python programming and jupyter (ipython notebook)
- the softwares mentioned in the [prerequesite](https://github.com/Population-Transcriptomics/C1-CAGE-preview/blob/master/prerequisite.md) section are all installed and referenced in your PATH
- you've already dowloaded the availabe [C1CAGE library](https://briefcase.riken.jp/public/HOSkQAul5gIANkcBn25PAJuWL3WpMERFuJD6C8JmjEgm) and extracted it in your curent directory
- The reference genome is already indexed with bwa

In our hands this notebook worked without trouble on a machine running Debian GNU/Linux 8. We noticed that the behavior of tagdust2 in single-end mode was different on Mac OSX. In short, the order of the reads1 is changed after extraction on Mac OSX which is a problem because syncpairs expect the order of reads1 and reads2 to be the same. One way to overcome this issueis sort reads1 and reads2 separately after the exctraction then syncpairs will work properly.

## Imports

In [1]:
import subprocess, os, csv, signal

## Custom functions

In [2]:
remove_extension = lambda x: x.split('.')[0]

Declare the function that deals with inputs and outputs

In [3]:
def get_args(read1, read2, ref_genome, output_folders):
    r1_shortname = remove_extension(os.path.basename(read1))

    args = {  
        'r1_input': read1,
        'r2_input': read2,
        'ref_genome': ref_genome,
    }
    
    output_paths = {folder: os.path.join('output', folder, r1_shortname) for folder in output_folders}
    
    return dict(args, **output_paths)

## Parameters

If the required softwares are not in the PATH you can manually set their location here

In [8]:
tagdust2_path = 'tagdust'
bwa_path = 'bwa'
samtools_path = 'samtools'
paired_bam_to_bed12_path = 'pairedBamToBed12'
umicountFP_path = 'umicountFP'
syncpairs_path = 'syncpairs'

Path to the reference genome you want to align your reads against

In [9]:
ref_genome = './GRCh38.fa'

In [10]:
softwares = {    
    'bwa': bwa_path,
    'tagdust': tagdust2_path,
    'syncpairs': syncpairs_path,
    'samtools': samtools_path,
    'pairedBamToBed12': paired_bam_to_bed12_path,
    'umicountFP': umicountFP_path}

The name of the output folders for each command

In [11]:
output_folders = ['tagdust_r1', 'unzip_r2', 'extracted_r1', 'extracted_r2', 'cleaned_reads', 'cleaned_r1', 'cleaned_r2', 
                  'r1_sai', 'r2_sai', 'sampe', 'sam_to_bam', 'genome_mapped',
                  'properly_paired', 'cagescan_pairs', 'cagescan_fragments']

Create the folders

In [13]:
for folder in output_folders:
    os.makedirs(os.path.join('output', folder))

The actual command to run. See the [tutorial](https://github.com/Population-Transcriptomics/C1-CAGE-preview/blob/master/tutorial.md) section for more details about each command

In [14]:
cmds = [
    
    '{tagdust} -t8 -o {tagdust_r1} -1 F:NNNNNNNN -2 S:TATAGGG -3 R:N {r1_input}',
    
    'gunzip -c {r2_input} > {unzip_r2}.fq',
        
    '{syncpairs} {tagdust_r1}.fq {unzip_r2}.fq {extracted_r1}.fq {extracted_r2}.fq',
    
    '{tagdust} -arch SimpleArchitecture.txt -ref hg38_rDNA.fa -o {cleaned_reads} {extracted_r1}.fq {extracted_r2}.fq',
    
    'cp {cleaned_reads}_READ1.fq {cleaned_r1}.fq',
    
    'cp {cleaned_reads}_READ2.fq {cleaned_r2}.fq',
    
    '{bwa} aln {ref_genome} {cleaned_r1}.fq > {r1_sai}.sai',
    
    '{bwa} aln {ref_genome} {cleaned_r2}.fq > {r2_sai}.sai',
    
    '{bwa} sampe -a 2000000 -c 0.00001 {ref_genome} {r1_sai}.sai {r2_sai}.sai {cleaned_r1}.fq {cleaned_r2}.fq > {sampe}.sam',
    
    '{samtools} view -bSo  {sam_to_bam}.bam {sampe}.sam',
    
    '{samtools} sort -n {sam_to_bam}.bam {genome_mapped}',
    
    '{samtools} view -f 0x0002 -F 0x0100 -bo {properly_paired}.bam {genome_mapped}.bam',
    
    '{pairedBamToBed12} -i {properly_paired}.bam > {cagescan_pairs}.bed',
    
    '{umicountFP} -f {cagescan_pairs}.bed > {cagescan_fragments}.bed'
    
]

Get the reads. Here we assume that the reads are in the current directory, in a folder named following the MiSeq run id

In [15]:
root, folders, files = os.walk('./150519_M00528_0125_000000000-ACUAB/').next()

files = [f for f in files if not f.startswith('.')] #remove hidden files if there exist
reads1 = sorted([os.path.join(root, f) for f in files if 'R1' in f])
reads2 = sorted([os.path.join(root, f) for f in files if 'R2' in f])

Run the commands for all the pairs

In [16]:
for read1, read2 in zip(reads1, reads2):
    args = get_args(read1, read2, ref_genome, output_folders)
    args = dict(args, **softwares)
    
    for cmd in cmds:
#         print cmd.format(**args)
        subprocess.call(cmd.format(**args), preexec_fn=lambda: signal.signal(signal.SIGPIPE, signal.SIG_DFL), shell=True)

Generate the level1 file

In [17]:
root, folders, files = os.walk('./output/genome_mapped/').next()
files = [os.path.join(root, f) for f in files if f.endswith('bam')]
level1 = 'python ./PromoterPipeline_20150516/level1.py -o output/mylevel1file.l1.osc.gz -f 0x40 --fingerprint {files}'.format(files=' '.join(files))

In [18]:
subprocess.call(level1, shell=True)

0

In [20]:
# root, folder, files = os.walk('output/tagdust_r1/').next()
# logs = [f for f in files if f.endswith('logfile.txt')]
# logs
# with open(os.path.join(root, logs[0])) as t2_log:
#     reader = csv.reader(t2_log, delimiter='\t')
#     rows = [row for row in reader]
#     non_empty = [row for row in rows if row]
#     spikes_rows = [row for row in non_empty if 'ERCC' in row[-1]]
#     number_of_spikes = sum([int(row[1]) for row in spikes_rows])
#     number_of_spikes